<a href="https://colab.research.google.com/github/dhan16/colabs/blob/master/ml/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LR: Model

##### Model
Given data of n observations {**x<sub>i</sub>**, y<sub>i</sub>}<sub>i=1:n</sub> with y<sub>i</sub> a scalar response and **x<sub>i</sub>** a column vector of size p:
* y<sub>i</sub> = β<sub>1</sub>x<sub>i1</sub> + ... + β<sub>p</sub>x<sub>ip</sub> + ε<sub>i</sub> = x<sub>i</sub><sup>T</sup>β + ε<sub>i</sub>

* or, in vector form: y<sub>i</sub> = **x<sub>i</sub>**<sup>T</sup>**β** + ε<sub>i</sub>

* or, stacking the n equations together in matrix notation: **y** = X**β** + **ε**
---

##### Ordinary Least Squares Solution
Find the coefficients **β** which fit the equations "best", **$\hat{β}$** = arg min L(**β**), where

* Loss function L(**β**) = ||**y** - X**β**||<sup>2</sup>

* The solution is: **$\hat{β}$** = (X<sup>T</sup>X)<sup>-1</sup> X<sup>T</sup>**y**

where (X<sup>T</sup>X)<sup>-1</sup> X<sup>T</sup> is  the Moore–Penrose pseudoinverse matrix of X

##### References
* https://en.wikipedia.org/wiki/Ordinary_least_squares#Linear_model
---


##### Gradient of the loss function

∇<sub>**β**</sub>L = 2X<sup>T</sup>(X**β** - **y**)

#### References
* https://www.inf.ed.ac.uk/teaching/courses/mlpr/2016/notes/w3b_regression_gradients.pdf
* https://d2l.ai/chapter_linear-regression/linear-regression.html#sec-linear-regression

### Utils

#### Data, np and plot utils

In [ ]:
# https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/validation_and_test_sets.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=validation_tf2-colab&hl=en
import numpy as np

def make_simple_regression_data(beta, n):
  np.random.seed(100) # seed random number generator
  x = 1000 * np.random.rand(n)
  e = 100 * np.random.rand(len(x))
  y = beta[0] + x*beta[1] + e
  print('make_simple_regression_data beta:', beta)
  return (np.array([[e] for e in x]), y)

from sklearn import datasets
def make_regression_data(n_samples, n_features, intercept=0):
  x, y, coef = datasets.make_regression(n_samples=n_samples,#number of samples
                                      n_features=n_features,#number of features
                                      n_informative=n_features,#number of useful features 
                                      bias=intercept,
                                      noise=10,#bias and standard deviation of the guassian noise
                                      coef=True,#true coefficient used to generated the data
                                      random_state=0) #set for sa, plot utilsts for each run
  if n_features == 1: # ugh, coef is a scalar for 1 feature, vector for > 1
    coef = [coef]
  beta = np.insert(coef, 0, intercept, axis=0)
  print('make_regression_data beta:', beta)
  return x, y, beta

In [ ]:
def scale(x, a, b):
  '''Scale numpy array x to be between a and b'''
  range = np.amax(x) - np.amin(x)
  return x / range * (b - a) + a

''' Returns matrix with first column as 1's from x, where x is an array of arrays'''
def add_intercept(x):
  x_plus=np.vstack((np.ones(len(x)), np.array(x).T)).T
  return x_plus

def remove_intercept(x):
  return x[:, 1:]

# Return true if the first column is all 1s
def has_intercept(x):
  return np.all(x[:, 0] == 1)

In [ ]:
import matplotlib.pyplot as plt
from typing import Dict

def lr_plot2D(x, y, beta):
  intercept = has_intercept(x)
  if intercept:
    x = remove_intercept(x)
  x = x.flatten()  # convert into 1D
  plt.plot(x, y, 'o', label='data')

  xx = np.linspace(min(x), max(y), 11)
  xx = x[:, np.newaxis] # convert into 2D
  yy = lr_predict_y(add_intercept(xx) if intercept else xx, beta)

  if intercept:
    label='fit, $y = {:.2f} + {:.2f} * x$'.format(beta[0], beta[1])
  else:
    label='fit, $y = {:.2f} * x$'.format(beta[0])
  plt.plot(xx, yy, label=label)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.legend(framealpha=1, shadow=True)
  plt.grid(alpha=0.25)
  plt.show()


def lr_plot3D(x, y, beta):
  # https://www.kaggle.com/code/spidy20/3d-visualization-of-multiple-linear-regression/notebook
  # https://gist.github.com/aricooperdavis/c658fc1c5d9bdc5b50ec94602328073b
  intercept = has_intercept(x)
  if intercept:
    x = remove_intercept(x)
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(111, projection='3d')
  ax.set_xlabel("X1")
  ax.set_ylabel("X2")
  ax.set_zlabel("y")
  ax.scatter(x[:,0], x[:,1], y, marker='.', color='red')

  x1_min, x1_max = min(x[:,0]), max(x[:,0])
  x2_min, x2_max = min(x[:,1]), max(x[:,1])
  xs = scale(np.tile(np.arange(101), (101,1)), x1_min, x1_max)
  ys = scale(np.tile(np.arange(101), (101,1)).T, x2_min, x2_max)
  if intercept:
    zs = beta[0] + xs*beta[1]+ ys*beta[2]
  else:
    zs = xs*beta[0]+ ys*beta[1]
  ax.plot_surface(xs,ys,zs, alpha=0.3)
  plt.show()


def plot_losscurves(losscurves: Dict):
  fmts = {
      'test_loss': 'g',
      'batch_loss': 'b',
      'train_loss_by_batch': 'r',
      'train_loss_by_epoch': 'r+',
  }
  for label, losscurve in losscurves.items():
    print(label, losscurve)
    plt.plot(losscurve, fmts[label], label=label)
  plt.legend(framealpha=1, shadow=True)
  plt.grid(alpha=0.25)
  plt.show()

#### LR Utils

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

''' x is an array of arrays, beta and y are arrays'''
def lr_predict_y(x, beta):
  return x.dot(beta)

# loss for current values of beta. we use mse so that batch and train/test errors are comparable
def lr_loss(X_mat, y, beta):
  return lr_sse(X_mat, y, beta) / len(y)

# sum of squared errors
def lr_sse(X_mat, y, beta):
  h = lr_predict_y(X_mat, beta)
  J = np.dot((h - y).transpose(), (h - y))
  return J

''' y and y_pred are arrays'''
def show_metrics(y, y_pred):
  print('mse: {:.4f} r2: {:.4f}'.format(mean_squared_error(y, y_pred), r2_score(y, y_pred)))

def lr_show_results(model_name, x, y, beta, metrics = True, plot = True):
  print('\n============ {} ============'.format(model_name))
  print('beta: {}'.format(beta))
  if metrics:
    show_metrics(y, lr_predict_y(x, beta))

  intercept = has_intercept(x)
  num_features = len(beta) - 1 if intercept else len(beta)
  if plot:
    if num_features == 1:
      lr_plot2D(x, y, beta)
    elif num_features == 2:
      lr_plot3D(x, y, beta)

### LR: Ordinary Least Squares method

##### OLS Impl

1. **$\hat{β}$** = (X<sup>T</sup>X)<sup>-1</sup> X<sup>T</sup>**y**. https://cmdlinetips.com/2020/03/linear-regression-using-matrix-multiplication-in-python-using-numpy/
2. **$\hat{β}$** = X<sup>pseudo-inverse</sup> **y**
3. scipy.linalg.lstsq https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html#scipy.linalg.lstsq
4. sklearn LinearRegression https://cmdlinetips.com/2020/03/linear-regression-using-matrix-multiplication-in-python-using-numpy/

In [ ]:
#@title OLS library
import numpy as np
from scipy.linalg import lstsq
from sklearn.linear_model import LinearRegression

# 1. https://cmdlinetips.com/2020/03/linear-regression-using-matrix-multiplication-in-python-using-numpy/
def ols_with_inverse(X_mat, y):
  return np.linalg.inv(X_mat.T.dot(X_mat)).dot(X_mat.T).dot(y)

# 2.
def ols_with_pseudoinverse(X_mat, y):
  return np.linalg.pinv(X_mat).dot(y)

# 3. https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html#scipy.linalg.lstsq
def ols_with_numpy_lstsq(X_mat, y):
  beta_hat, res, rnk, s = lstsq(X_mat, y)
  return beta_hat
  
# 4. sklearn.linear_model.LinearRegression https://cmdlinetips.com/2020/03/linear-regression-using-matrix-multiplication-in-python-using-numpy/
def ols_with_sklearn(X_mat, y, fit_intercept=False):
  lr = LinearRegression(fit_intercept=fit_intercept).fit(x, y)
  if fit_intercept:
    beta_hat = np.insert(lr.coef_, 0, lr.intercept_, axis=0)
  else:
    beta_hat = lr.coef_
  return beta_hat

def run_and_show_olss(x, y, inv=False, pinv=False, lstsq=False, sklearn=True, fit_intercept=False):
  if fit_intercept:
    x=add_intercept(x)
  if inv:
    lr_show_results('ols_with_inverse', x, y, ols_with_inverse(x, y))
  if pinv:
    lr_show_results('ols_with_pseudoinverse', x, y, ols_with_pseudoinverse(x, y))
  if lstsq:
    lr_show_results('ols_with_numpy_lstsq', x, y, ols_with_numpy_lstsq(x, y))
  if sklearn:
    lr_show_results('ols_with_sklearn', x, y, ols_with_sklearn(x, y, fit_intercept=fit_intercept))

##### OLS Runs

In [ ]:
(x, y) = make_simple_regression_data([50, 3], 100)
run_and_show_olss(x, y, inv=True, pinv=True, lstsq=True, sklearn=True, fit_intercept=True)

In [ ]:
run_and_show_olss(x, y, inv=True, pinv=True, lstsq=True, sklearn=True, fit_intercept=False)

In [ ]:
x, y, coef = make_regression_data(n_samples=100, n_features=1, intercept=100)
run_and_show_olss(x, y, fit_intercept=True)

In [ ]:
run_and_show_olss(x, y, fit_intercept=False)

In [ ]:
x, y, coef = make_regression_data(n_samples=1000, n_features=2, intercept=300)
run_and_show_olss(x, y, fit_intercept=True)

In [ ]:
run_and_show_olss(x, y, fit_intercept=False)


### LR: Gradient descent



#### LR GradD: Common functions

In [ ]:
#@title Gradient Descent library
from sklearn.utils import shuffle
from sklearn.utils import gen_batches

# function to perform mini-batch gradient descent
'''x is an array of arrays'''
def gradient_descent(x, y, grad_fn, loss_fn, 
                     epochs=3, batch_size=32, learning_rate=0.001,
                     model='Gradient Descent'):
  print('{} epochs:{} batch_size:{}, learning_rate:{}'.format(model, epochs, batch_size, learning_rate))
  beta = np.zeros(x.shape[1])
  batch_losses = []
  train_losses_by_batch = []
  train_losses_by_epoch = []
  for itr in range(epochs):
    x, y = shuffle(x, y)
    for batch_slice in gen_batches(len(y), batch_size):
      x_mini, y_mini = x[batch_slice], y[batch_slice]
      beta = beta - learning_rate * grad_fn(x_mini, y_mini, beta)  
      batch_losses.append(loss_fn(x_mini, y_mini, beta))
      train_losses_by_batch.append(loss_fn(x, y, beta))
    train_losses_by_epoch.append(loss_fn(x, y, beta))    

 
  losscurves = {
      'batch_loss': batch_losses,
      'train_loss_by_batch' : train_losses_by_batch,
      'train_loss_by_epoch': train_losses_by_epoch
  }
  return beta, losscurves

#### LR GradD: Analytical gradient

##### LR GradD: Analytical gradient Impl

* Gradient of the loss function: ∇<sub>**β**</sub>L = 2X<sup>T</sup>(X**β** - **y**)
* https://www.geeksforgeeks.org/ml-mini-batch-gradient-descent-with-python/


In [ ]:
#@title

# function to compute gradient for regression loss function
def lr_grad(X_mat, y, beta):
  h = lr_predict_y(X_mat, beta)
  grad = np.dot(X_mat.transpose(), (h - y))
  return grad


from sklearn.utils import shuffle
from sklearn.utils import gen_batches
# function to perform mini-batch gradient descent
'''x is an array of arrays without the intercept term'''
def lr_gradient_descent(x, y, epochs=3, batch_size=32, learning_rate=0.001):
  return gradient_descent(x, y, lr_grad, lr_loss, 
                          epochs=epochs, batch_size=batch_size, learning_rate=learning_rate,
                          model='LR GradD: Analytical Gradient')

def run_and_show_lrgd(x, y, **kwargs):
  x=add_intercept(x)
  beta, losscurves = lr_gradient_descent(x, y, **kwargs)
  lr_show_results('LR GradD: Analytical Gradient', x, y, beta)
  plot_losscurves(losscurves)

##### LR GradD: Analytical gradient Runs

In [ ]:
x, y, coef = make_regression_data(n_samples=100, n_features=1)
run_and_show_lrgd(x, y, batch_size=32, epochs=20, learning_rate=.01)
run_and_show_olss(x, y)

In [ ]:
x, y, coef = make_regression_data(n_samples=1000, n_features=10, intercept=3)
# run_and_show_lrgd(x, y)
run_and_show_lrgd(x, y, batch_size=32, epochs=10, learning_rate=.001)
run_and_show_olss(x, y)

#### LR GradD: Forward-mode auto-diff


##### Multivariate chain rule

The crux of autodiff is the multivariate chain rule. Suppose we have
* a function f in two variables
* we want to compute $\dfrac{\mathrm{d}f(x(t),y(t))}{\mathrm{d}t}$

Changing t slightly has two effects: it changes x slightly,
and it changes y slightly. Each of these effects causes a slight change to f. For infinitesimal changes, these effects combine additively. The Chain Rule is therefore:

$\dfrac{\mathrm{d}f(x(t),y(t))}{\mathrm{d}t} = \dfrac{∂f}{∂x} \dfrac{dx}{dt} + \dfrac{∂f}{∂y} \dfrac{dy}{dt}$

##### References
* https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/readings/L06%20Backpropagation.pdf
* https://en.wikipedia.org/wiki/Chain_rule#Multivariable_case


##### Forward mode

We are going to calculate the function evaluation and its derivative for an arbitrary function f given some data x using operator overloading i.e. we are going to change the default behavior of the core arithmetic operators such as +, -, /, * and the mathematical functions max, min, mean, log, cos, sin, and so-on, for a custom class.

We create a class Variable which keeps track of function evaluation and the derivative using the chain rule.

##### References
*  https://mdrk.io/introduction-to-automatic-differentiation/
* https://mdrk.io/introduction-to-automatic-differentiation-part2/

##### Implementation

In [ ]:
#@title Forward mode auto-diff library

class Variable():
  def __init__(self, f, d = 0):
    self.f = f
    self.d = d
    
  def __repr__(self):
    return f"Variable(f={self.f}, d={self.d})"
    
  def __add__(self, other): # other + self
    return Variable(self.f + other.f, self.d + other.d)

  def __sub__(self, other): # self - other
    return Variable(self.f - other.f, self.d - other.d)

  def __mul__(self, other): # self * other
    '''uses product rule'''
    return Variable(self.f * other.f, other.f * self.d + other.d * self.f)

  def __truediv__(self, other): # self / other
    '''uses differentiation rule for div'''
    return Variable(self.f / other.f, (self.d * other.f - self.f * other.d) / other.f**2)

  def __radd__(self, other): # other + self
    return other + self
  
  def __rsub__(self, other): # other - self
    return other - self

  def __rmul__(self, other): # other * self
    return other * self

  def __rtruediv__(self, other): # other / self
    return other/self

In [ ]:
def f(x, a, b):
  return x * a / (x * x + b)

# x=24, a=22,b=2
x = Variable(24, 1) # d = 1 since derivatve of x wrt x is 1
a = Variable(22, 0) # d = 0 since derivatve of a wrt x is 0
b = Variable(2, 0) # d = 0 since derivatve of b wrt x is 0

print(f(x, a, b))

#### LR GradD: JAX


References

* https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/validation_and_test_sets.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=validation_tf2-colab&hl=en#scrollTo=FBhNIdUatOU6

* https://colab.research.google.com/github/kaustubholpadkar/Predicting-House-Price-using-Multivariate-Linear-Regression/blob/master/Multivariate_Linear_Regression_Python.ipynb

* https://www.coursera.org/projects/regression-automatic-differentiation-tensorflow



In [ ]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x

In [ ]:
#@title Import modules
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

### LR: Tensorflow


### ML

#### Feature Engineering

* Load data
* Shuffle
* Split into training, validation and test. 
* Scale label column to meaningful values

Features
* Bucketize, cross 
* Scale features - min max or z scale


References

* https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/representation_with_a_feature_cross.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=representation_tf2-colab&hl=en


#### Define and Train Models
* create model
* train model
* plot loss curves
